In [1]:
# Imports

import pandas as pd
import biogeme.biogeme as bio
import biogeme.database as db
from biogeme import models
from biogeme.expressions import Beta
import numpy as np

In [2]:
# initialising data
df = pd.read_csv('Dataset 1 2024.csv')
df.rename(columns={'Unnamed: 0':'ID','CAR OWNERSHIP':'CAR_OWNERSHIP'}, inplace=True)
db = db.Database("Dataset 1 2024",df)

In [3]:
## creating the model parameters

# car alternative specific constants
Beta0_car = Beta('Beta0_car', 0, None, None, 0)
BetaTT_car = Beta('BetaTT_car', 0, None, None, 0)
BetaFuel_car = Beta('BetaFuel_car', 0, None, None, 0)
BetaParking_car = Beta('BetaParking_car', 0, None, None, 0)
BetaCongestion_car = Beta('BetaCongestion_car', 0, None, None, 0)
BetaOwnership_car = Beta('BetaOwnership_car', 0, None, None, 0)

# public transport alternative specific constants
Beta0_PT = Beta('Beta0_PT', 0, None, None, 0)
BetaTT_PT = Beta('BetaTT_PT', 0, None, None, 0)
BetaFare_PT = Beta('BetaFare_PT', 0, None, None, 0)
BetaAC_PT = Beta('BetaAC_PT', 0, None, None, 0)
BetaAT_PT = Beta('BetaAT_PT', 0, None, None, 0)
BetaWT_PT = Beta('BetaWT_PT', 0, None, None, 0)


# cycling alternative specific constants
Beta0_cycling = Beta('Beta0_cycling', 0, None, None, 0)
BetaTT_cycling = Beta('BetaTT_cycling', 0, None, None, 0)
BetaBikeability_cycling = Beta('BetaBikeability_cycling', 0, None, None, 0)

# walking alternative specific constants
Beta0_walking = Beta('Beta0_walking', 0, None, None, 1)
BetaTT_walking = Beta('BetaTT_walking', 0, None, None, 0)
BetaWalkability_walking = Beta('BetaWalkability_walking', 0, None, None, 0)

# demographic attribute generic parameters
BetaAge = Beta('BetaAge', 0, None, None, 0)
BetaGender = Beta('BetaGender', 0, None, None, 0)


In [ ]:
globals().update(db.variables)

In [5]:
# baseline utility functions
baseline_user = BetaAge * AGE + BetaGender * GENDER

V_baseline_car = Beta0_car + BetaTT_car * TT_CAR + BetaFuel_car * FC_CAR + BetaParking_car * PC_CAR + BetaCongestion_car * CC_CAR + BetaOwnership_car * CAR_OWNERSHIP + baseline_user
V_baseline_PT = Beta0_PT + BetaTT_PT * TT_PT + BetaFare_PT * FARE_PT + BetaAC_PT * AC_PT + BetaAT_PT * AT_PT + BetaWT_PT * WT_PT + baseline_user
V_baseline_cycling = Beta0_cycling + BetaTT_cycling * TT_CYCLE + BetaBikeability_cycling * BIKEABILITY_INDEX + baseline_user
V_baseline_walking = Beta0_walking + BetaTT_walking * TT_WALK + BetaWalkability_walking * WALKABILITY_INDEX + baseline_user

In [6]:
# intervention 1, AC_PT = 0
V_intervention1_PT = Beta0_PT + BetaTT_PT * TT_PT + BetaFare_PT * FARE_PT + BetaAT_PT * AT_PT + BetaWT_PT * WT_PT + baseline_user

# intervention 2, AT_PT*reduction
reductionFactor = 0.5 # change###################################################
V_baseline_PT = Beta0_PT + BetaTT_PT * TT_PT + BetaFare_PT * FARE_PT + BetaAC_PT * AC_PT + BetaAT_PT * AT_PT*reductionFactor + BetaWT_PT * WT_PT + baseline_user


In [7]:
# associate utility functions with the numbering of alternatives
V_base = {4: V_baseline_car, 3: V_baseline_PT, 2: V_baseline_cycling, 1: V_baseline_walking}
V_intervention1 = {4: V_baseline_car, 3: V_intervention1_PT, 2: V_baseline_cycling, 1: V_baseline_walking}
V_intervention2 = {4: V_baseline_car, 3: V_baseline_PT, 2: V_baseline_cycling, 1: V_baseline_walking}
av = {4: 1, 3: 1, 2: 1, 1: 1}

In [8]:
# logit model
logprob_base = models.loglogit(V_base, av, CHOSEN_MODE)
logprob_intervention1 = models.loglogit(V_intervention1, av, CHOSEN_MODE)
logprob_intervention2 = models.loglogit(V_intervention2, av, CHOSEN_MODE)

In [ ]:
# initialising the models
baseline = bio.BIOGEME(db, logprob_base)
intervention1 = bio.BIOGEME(db, logprob_intervention1)
intervention2 = bio.BIOGEME(db, logprob_intervention2)

baseline.modelName = "baseline_model"
intervention1.modelName = "intervention1_model"
intervention2.modelName = "intervention2_model"

File biogeme.toml has been created


In [ ]:
# baseline model estimation
results_baseline = baseline.estimate()
results_baseline.getEstimatedParameters()

C:\Users\majdy\AppData\Local\Temp\ipykernel_38212\537661990.py:2: DeprecationWarning: getEstimatedParameters is deprecated; use get_estimated_parameters instead.
  results_baseline.getEstimatedParameters()


,Value,Rob. Std err,Rob. t-test,Rob. p-value
Beta0_PT,3.366397e+00,3.262284e-01,10.319143,0.000000e+00
Beta0_car,3.057305e+00,1.750033e-01,17.469992,0.000000e+00
Beta0_cycling,6.936734e-02,9.855690e-02,0.703830,4.815384e-01
BetaAC_PT,-5.670327e-01,4.438097e-02,-12.776481,0.000000e+00
BetaAT_PT,-1.942133e-01,1.884135e-02,-10.307824,0.000000e+00
BetaAge,2.369965e-13,8.923249e-14,2.655944,7.908665e-03
BetaBikeability_cycling,5.339313e-03,8.614533e-04,6.198030,5.717413e-10
BetaCongestion_car,-1.293574e-01,1.293177e-02,-10.003068,0.000000e+00
BetaFare_PT,-1.138719e+00,6.905384e-02,-16.490304,0.000000e+00
BetaFuel_car,-5.525109e-01,3.991692e-02,-13.841521,0.000000e+00


In [ ]:
# intervention 1 model estimation
results_intervention1 = intervention1.estimate()
results_intervention1.getEstimatedParameters()

C:\Users\majdy\AppData\Local\Temp\ipykernel_38212\1587028504.py:2: DeprecationWarning: getEstimatedParameters is deprecated; use get_estimated_parameters instead.
  results_intervention1.getEstimatedParameters()


,Value,Rob. Std err,Rob. t-test,Rob. p-value
Beta0_PT,1.837482e+00,2.775167e-01,6.621161e+00,3.563905e-11
Beta0_car,3.052316e+00,1.747786e-01,1.746390e+01,0.000000e+00
Beta0_cycling,7.335185e-02,9.828855e-02,7.462909e-01,4.554917e-01
BetaAT_PT,-9.411753e-02,9.096498e-03,-1.034657e+01,0.000000e+00
BetaAge,9.464806e-14,1.797693e+308,5.286502e-322,1.000000e+00
BetaBikeability_cycling,5.299104e-03,8.590780e-04,6.168363e+00,6.900076e-10
BetaCongestion_car,-1.288074e-01,1.290419e-02,-9.981825e+00,0.000000e+00
BetaFare_PT,-1.046211e+00,6.153939e-02,-1.700068e+01,0.000000e+00
BetaFuel_car,-5.492123e-01,3.979560e-02,-1.380083e+01,0.000000e+00
BetaGender,1.093337e-16,1.797693e+308,0.000000e+00,1.000000e+00


In [ ]:
# intervention 2 model estimation
results_intervention2 = intervention2.estimate()
results_intervention2.getEstimatedParameters()

C:\Users\majdy\AppData\Local\Temp\ipykernel_38212\1794683177.py:2: DeprecationWarning: getEstimatedParameters is deprecated; use get_estimated_parameters instead.
  results_intervention2.getEstimatedParameters()


,Value,Rob. Std err,Rob. t-test,Rob. p-value
Beta0_PT,3.362953e+00,3.261260e-01,10.311821,0.000000e+00
Beta0_car,3.057369e+00,1.750179e-01,17.468890,0.000000e+00
Beta0_cycling,6.767150e-02,9.855595e-02,0.686630,4.923157e-01
BetaAC_PT,-5.667812e-01,4.436853e-02,-12.774397,0.000000e+00
BetaAT_PT,-1.941353e-01,1.883808e-02,-10.305470,0.000000e+00
BetaAge,-7.992881e-14,2.502609e-14,-3.193819,1.404041e-03
BetaBikeability_cycling,5.345641e-03,8.614517e-04,6.205388,5.456222e-10
BetaCongestion_car,-1.293932e-01,1.293238e-02,-10.005367,0.000000e+00
BetaFare_PT,-1.138448e+00,6.903020e-02,-16.492025,0.000000e+00
BetaFuel_car,-5.526369e-01,3.991904e-02,-13.843941,0.000000e+00


In [13]:
results_baseline.write_latex()
results_intervention1.write_latex()
results_intervention2.write_latex()